In [53]:
from sklearn import svm,metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import glob, os.path, re

matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

html 파일 변환 및 pandas data로 불러오기

In [54]:
# url 내용을 csv 파일로 저장하고, df로 변환 후 출력하시오.
import requests
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
res = requests.get(url)
res.raise_for_status()

# 파일 저장
with open('a.csv', 'w', encoding='utf-8-sig') as f:
    f.write(res.text)
    
df = pd.read_csv('./a.csv', header=None)
df 

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


독버섯과 식용버섯을 분류하는 알고리즘을 구현하시오.
데이터를 보면
- p : 독버섯
- e : 식용버섯

In [55]:
df.shape

(8124, 23)

In [56]:
df.iloc[:,1:].head(3)

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m


In [57]:
df.iterrows()

<generator object DataFrame.iterrows at 0x00000209527D9F10>

In [58]:
target = []
data = []
# attr_list = []

In [59]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m


In [60]:
df.loc[0]
df.loc[0, 1:] # 첫번째 행에서 첫번째 열 값 제외한 나머지 열들의 값

1     x
2     s
3     n
4     t
5     p
6     f
7     c
8     n
9     k
10    e
11    e
12    s
13    s
14    w
15    w
16    p
17    w
18    o
19    p
20    k
21    s
22    u
Name: 0, dtype: object

In [61]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g


In [62]:
# # 데이터 전처리
# 아스키코드로 변환(문자 -> 숫자 변환 코드)
# for row_index, row in df.iterrows():
#     target.append(row.loc[0]) # target 결과 데이터 저장
#     row_data = []
#     for v in row.loc[1:] :
#         row_data.append(ord(v)) # 1줄의 데이터를 숫자로 변경해서 list에 저장
    
#     data.append(row_data) # 1줄 리스트를 다시 리스트에 추가

# data

# df2 = pd.DataFrame(data)
# df2.head() # 숫자로 변경해야 머신러닝 가능

In [64]:
# row.loc[:, 1:]
# x	 s	n	t	p	f	c	n	k	...	s	w	w	p	w	o	p	k	s	u

In [ ]:
# 데이터 전처리
# 원핫 인코딩으로 변환(문자 -> 숫자 변환 코드)
target = []
data = []
attr_list = []
for row_index, row in df.iterrows():
    target.append(row.loc[0]) # target 결과 데이터 저장
    row_data = []
    for col, v in enumerate(row.loc[1:]) : # v는 22개 데이터 넘어와
        if row_index == 0:
            attr = {"dic":{}, "cnt":0} # cnt 종류의 수
            attr_list.append(attr)
        else :
            attr = attr_list[col]
            
        # 원핫인코딩 변환
        d = [0]*12 # data 안의 알파벳이 12개이므로 12 # [0,0,0,0,0,0,0,0,0,0,0,0]
        if v in attr['dic']:
            idx = attr['dic'][v]
            # print('idx : ', idx)
        else : 
            idx = attr['cnt']
            attr['dic'][v] = idx
            # print('attr["dic"][v] : ', attr['dic'][v])
            attr['cnt'] += 1
            
        d[idx] = 1
        row_data += d # 원핫인코딩의 데이터 저장 파일, 데이터의 가중치 때문에 정답률이 다를 수 있음
    
    data.append(row_data) # 1줄 리스트를 다시 리스트에 추가

data[0]
# df2 = pd.DataFrame(data)
# df2.head() # 숫자로 변경해야 머신러닝 가능

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Regression
- 
- 
Desicion Tree
Random forest
Classfication 
- KNN
- Trees
- Logistic Regression
- Naice-Bayes
- SVM

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 데이터 전처리 : 위의 코드 확인
  # train, test 세트 구현
train_input, test_input, train_target, test_target = train_test_split(data, target)

# 데이터 학습시키기 
# clf = RandomForestClassifier() # score : 1.0
clf = svm.SVC() # score : 0.9965534219596258 # 원핫 인코딩변환 후 1.0, 데이터의 가중치 때문에 정답률이 다를 수 있음
# clf = KNeighborsClassifier() # score : 1.0
clf.fit(train_input, train_target)

# 데이터 예측하기
predict = clf.predict(test_input)
print('결과값 : ', predict)

# 정답률 구하기
score = clf.score(test_input, test_target)
print('정답률 : ', score)

결과값 :  ['e' 'p' 'e' ... 'p' 'p' 'p']
정답률 :  1.0


In [ ]:
# data = df.iloc[0,1:]
data = df.iloc[:, 1:] # 전체
# d_list = []
all_list = [] # [[],[],[],[],...]
for d in data :
    d_list.append(ord(d))
d_list

[120,
 115,
 110,
 116,
 112,
 102,
 99,
 110,
 107,
 101,
 101,
 115,
 115,
 119,
 119,
 112,
 119,
 111,
 112,
 107,
 115,
 117]

In [ ]:
df # 0번째 열이 target, 나머지 열이 input
# 데이터분석은 숫자만 돼. 데이터는 문자이므로 숫자로 변환 필요
df.iloc[:,:23] # ==> 숫자코드로 변환
df.iloc[:,1:]

# mc
# cnt = [ 0 for n in range(0, 26) ]
# code_a = ord('a')
# code_z = ord('z')

# for d in df :
#     n = ord(d)
#     if code_a <= n <= code_z:
#         cnt[n-code_a] += 1

# total = sum(cnt)
# freq = list(map(lambda n: n/total, cnt))

# print()

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,s,o,o,p,o,o,p,b,c,l
8120,x,s,n,f,n,a,c,b,y,e,...,s,o,o,p,n,o,p,b,v,l
8121,f,s,n,f,n,a,c,b,n,e,...,s,o,o,p,o,o,p,b,c,l
8122,k,y,n,f,y,f,c,n,b,t,...,k,w,w,p,w,o,e,w,v,l
